<a href="https://colab.research.google.com/github/rishabhsai/AI_code_debaterr/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Python libraries (including PDF generation)
!pip install -q -U openai google-generativeai gradio reportlab markdown

# 2. Import libraries and configure clients
import openai
import google.generativeai as genai
import gradio as gr
from google.colab import userdata
import io
import base64
from datetime import datetime
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Preformatted
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.colors import HexColor
import re

# Initialize clients to None
openai_client = None
gemini_model = None

# Configure OpenAI client
try:
    api_key = userdata.get('OPENAI_API_KEY')
    openai_client = openai.OpenAI(api_key=api_key)
    print("✅ OpenAI (GPT-4o mini) client configured successfully.")
except Exception as e:
    print(f"❌ OpenAI setup failed: {e}")
    print("Make sure you have added 'OPENAI_API_KEY' to Colab Secrets (🔑)")

# Configure Gemini client
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    gemini_model = genai.GenerativeModel('gemini-1.5-flash')
    print("✅ Gemini client configured successfully.")
except Exception as e:
    print(f"❌ Gemini setup failed: {e}")
    print("Make sure you have added 'GEMINI_API_KEY' to Colab Secrets (🔑)")

print("\n🚀 Setup complete! Run the next cell to launch the app.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 6.9 MB/s eta 0:00:00
✅ OpenAI (GPT-4o mini) client configured successfully.
✅ Gemini client configured successfully.

🚀 Setup complete! Run the next cell to launch the app.


In [ ]:
def create_pdf_report(debate_log, final_code, user_prompt):

    # Creates a PDF report of the entire debate conversation including all reviewer notes

    buffer = io.BytesIO()

    # Create the PDF document
    doc = SimpleDocTemplate(buffer, pagesize=A4,
                          rightMargin=72, leftMargin=72,
                          topMargin=72, bottomMargin=18)

    # Get styles
    styles = getSampleStyleSheet()

    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        textColor=HexColor('#2E86AB'),
        alignment=1  # Center alignment
    )

    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        fontSize=16,
        spaceAfter=12,
        textColor=HexColor('#A23B72'),
        spaceBefore=20
    )

    subheading_style = ParagraphStyle(
        'CustomSubHeading',
        parent=styles['Heading3'],
        fontSize=14,
        spaceAfter=8,
        spaceBefore=12,
        textColor=HexColor('#1F4E79')
    )

    code_style = ParagraphStyle(
        'CodeStyle',
        parent=styles['Code'],
        fontSize=9,
        leftIndent=20,
        backgroundColor=HexColor('#F8F8F8'),
        borderColor=HexColor('#DDDDDD'),
        borderWidth=1,
        borderPadding=8,
        fontName='Courier'
    )

    review_style = ParagraphStyle(
        'ReviewStyle',
        parent=styles['Normal'],
        fontSize=11,
        leftIndent=15,
        rightIndent=15,
        backgroundColor=HexColor('#FFF9E6'),
        borderColor=HexColor('#FFD700'),
        borderWidth=1,
        borderPadding=10,
        spaceBefore=5,
        spaceAfter=10
    )

    # list to hold all elements
    story = []

    # Title
    story.append(Paragraph("🤖💎 AI Code Generation Debate Report", title_style))
    story.append(Spacer(1, 20))

    # Metadata
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    story.append(Paragraph(f"<b>Generated:</b> {timestamp}", styles['Normal']))
    story.append(Paragraph(f"<b>Models:</b> GPT-4o mini (Coder) vs Gemini 1.5 Flash (Reviewer)", styles['Normal']))
    story.append(Spacer(1, 20))

    # User Request
    story.append(Paragraph("📝 User Request", heading_style))
    story.append(Paragraph(user_prompt, styles['Normal']))
    story.append(Spacer(1, 20))

    # Parse the debate log more carefully
    lines = debate_log.split('\n')
    current_section = None
    current_content = []

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Check for round headers
        if line.startswith('--- **Round') and line.endswith('** ---'):
            # Process previous section if exists
            if current_section and current_content:
                process_section(story, current_section, current_content, code_style, review_style, subheading_style, styles)

            # Start new round
            round_match = re.search(r'Round (\d+)', line)
            if round_match:
                round_num = round_match.group(1)
                story.append(Paragraph(f"🔄 Round {round_num}", heading_style))
                story.append(Spacer(1, 10))
            current_section = None
            current_content = []

        # Check for coder section
        elif line.startswith('**🤖 Coder Agent (GPT-4o mini):**'):
            # Process previous section if exists
            if current_section and current_content:
                process_section(story, current_section, current_content, code_style, review_style, subheading_style, styles)

            current_section = 'coder'
            current_content = []

        # Check for reviewer section
        elif line.startswith('**💎 Reviewer Agent (Gemini):**'):
            # Process previous section if exists
            if current_section and current_content:
                process_section(story, current_section, current_content, code_style, review_style, subheading_style, styles)

            current_section = 'reviewer'
            current_content = []

        # Check for agreement
        elif 'Agreement reached' in line:
            # Process previous section if exists
            if current_section and current_content:
                process_section(story, current_section, current_content, code_style, review_style, subheading_style, styles)

            story.append(Paragraph("🎉 Agreement Reached!", subheading_style))
            story.append(Paragraph("The reviewer is satisfied with the code.", styles['Normal']))
            story.append(Spacer(1, 15))
            current_section = None
            current_content = []

        # Collect content for current section
        elif current_section and line:
            current_content.append(line)

        i += 1

    # Process final section if exists
    if current_section and current_content:
        process_section(story, current_section, current_content, code_style, review_style, subheading_style, styles)

    # Final Code Section
    story.append(Paragraph("✅ Final Agreed-Upon Code", heading_style))
    story.append(Preformatted(final_code, code_style))

    # Build PDF
    doc.build(story)

    # Get the PDF data
    buffer.seek(0)
    pdf_data = buffer.getvalue()
    buffer.close()

    return pdf_data

def process_section(story, section_type, content, code_style, review_style, subheading_style, styles):
    """
    Helper function to process different sections of the debate
    """
    if section_type == 'coder':
        story.append(Paragraph("🤖 Coder Agent (GPT-4o mini)", subheading_style))

        # Look for code blocks
        content_text = '\n'.join(content)
        if '```python' in content_text:
            # Extract code from markdown
            code_match = re.search(r'```python\n(.*?)\n```', content_text, re.DOTALL)
            if code_match:
                code = code_match.group(1)
                story.append(Preformatted(code, code_style))
        else:
            # If no code blocks, treat as regular text
            clean_content = '\n'.join(content).replace('```python', '').replace('```', '')
            if clean_content.strip():
                story.append(Preformatted(clean_content.strip(), code_style))

        story.append(Spacer(1, 15))

    elif section_type == 'reviewer':
        story.append(Paragraph("💎 Reviewer Agent (Gemini) - Feedback", subheading_style))

        # Clean up reviewer content
        review_text = '\n'.join(content)
        # Remove any markdown formatting
        review_text = review_text.replace('**', '').replace('*', '').strip()

        if review_text:
            # Split into paragraphs for better formatting
            paragraphs = review_text.split('\n\n')
            for para in paragraphs:
                if para.strip():
                    story.append(Paragraph(para.strip(), review_style))

        story.append(Spacer(1, 15))

def generate_code_with_debate(user_prompt, rounds=3):
    # Check if clients are configured
    if not openai_client or not gemini_model:
        error_msg = "❌ Client setup failed. Please check the setup cell output and try again."
        return "Error: Clients not configured.", error_msg, None

    debate_log = f"**User Request:**\n{user_prompt}\n\n"
    print(f"Starting debate for: {user_prompt}")

    coder_system_prompt = """
    You are an expert Python programmer (Coder Agent). Your sole purpose is to
    write clean, efficient, and correct Python code based on the user's request
    and the reviewer's feedback. Provide only the raw code without any explanations
    or markdown formatting. Just return the pure Python code.
    """

    reviewer_system_prompt = """
    You are a meticulous and critical code reviewer (Reviewer Agent). Your job
    is to find flaws in the provided Python code. Check for bugs, edge cases,
    non-standard practices, and areas for improvement in performance or
    readability. Provide your feedback as a clear, concise list. If the code
    is perfect, respond with "The code looks good. No changes needed."
    """

    current_code = "No code generated yet."

    # OpenAI conversation history
    coder_messages = [
        {"role": "system", "content": coder_system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    for i in range(rounds):
        round_num = i + 1
        debate_log += f"--- **Round {round_num}** ---\n\n"
        print(f"--- Round {round_num} ---")

        # Coder's turn (GPT-4o mini)
        try:
            print("🤖 Coder (GPT-4o mini) is thinking...")
            coder_response = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=coder_messages,
                temperature=0.7
            )
            current_code = coder_response.choices[0].message.content.strip()

            # Clean up code block formatting if present
            if "```python" in current_code:
                current_code = current_code.split("```python\n")[1].split("```")[0]
            elif "```" in current_code:
                current_code = current_code.split("```")[1].split("```")[0]

            debate_log += f"**🤖 Coder Agent (GPT-4o mini):**\n```python\n{current_code}\n```\n\n"

            # Add to conversation history
            coder_messages.append({"role": "assistant", "content": current_code})

        except Exception as e:
            debate_log += f"**❌ Error calling Coder Agent:** {e}\n"
            print(f"Error with Coder: {e}")
            break

        # Reviewer's turn (Gemini)
        try:
            print("💎 Reviewer (Gemini) is thinking...")

            # Create the review prompt for Gemini
            review_prompt = f"{reviewer_system_prompt}\n\nPlease review this Python code:\n\n{current_code}"

            reviewer_response = gemini_model.generate_content(review_prompt)
            review = reviewer_response.text.strip()

            debate_log += f"**💎 Reviewer Agent (Gemini):**\n{review}\n\n"

        except Exception as e:
            debate_log += f"**❌ Error calling Reviewer Agent:** {e}\n"
            print(f"Error with Reviewer: {e}")
            break

        # Check for agreement
        if "no changes needed" in review.lower() or "looks good" in review.lower():
            debate_log += "**🎉 Agreement reached!** The reviewer is satisfied with the code."
            print("🎉 Agreement reached!")
            break

        # Prepare for Coder's next turn
        coder_messages.append({
            "role": "user",
            "content": f"Please revise the code based on this review feedback:\n\n{review}"
        })

    print("✅ Debate finished.")

    # Generate PDF
    try:
        pdf_data = create_pdf_report(debate_log, current_code, user_prompt)
        # Create a temporary file for download
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"ai_code_debate_{timestamp}.pdf"

        # Save to a temporary location that Gradio can access
        with open(filename, 'wb') as f:
            f.write(pdf_data)

        return current_code, debate_log, filename
    except Exception as e:
        print(f"PDF generation error: {e}")
        return current_code, debate_log, None

# Create the Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="AI Code Debate") as demo:
    gr.Markdown(
        """
        # 🤖💎 AI Code Generation Debate

        **GPT-4o mini** (Coder) vs **Gemini 1.5 Flash** (Reviewer)

        Watch two different AI models collaborate! GPT-4o mini will write the initial code,
        and Gemini will review and suggest improvements through multiple rounds of discussion.
        """
    )

    with gr.Row():
        user_input = gr.Textbox(
            label="💭 Your Code Request",
            placeholder="e.g., Write a Python function to find all prime numbers up to n using the Sieve of Eratosthenes",
            lines=3,
        )

    with gr.Row():
        rounds_slider = gr.Slider(
            minimum=1,
            maximum=5,
            value=3,
            step=1,
            label="🔄 Number of Debate Rounds"
        )
        submit_button = gr.Button("🚀 Generate Code", variant="primary", size="lg")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📜 Debate Log")
            log_output = gr.Markdown()
        with gr.Column(scale=1):
            gr.Markdown("### ✅ Final Agreed-Upon Code")
            code_output = gr.Code(language="python")

    # PDF Download Section
    with gr.Row():
        pdf_download = gr.File(label="📄 Download Debate Report (PDF)", visible=False)

    def handle_debate_and_pdf(user_prompt, rounds):
        code, log, pdf_file = generate_code_with_debate(user_prompt, rounds)
        if pdf_file:
            return code, log, gr.File(value=pdf_file, visible=True)
        else:
            return code, log, gr.File(visible=False)

    submit_button.click(
        fn=handle_debate_and_pdf,
        inputs=[user_input, rounds_slider],
        outputs=[code_output, log_output, pdf_download],
    )

    # Add some example prompts
    gr.Examples(
        examples=[
            ["Write a Python function to calculate fibonacci numbers recursively"],
            ["Create a class for a simple calculator with basic operations"],
            ["Write a function to validate email addresses using regex"],
            ["Implement a binary search algorithm"],
            ["Create a function to merge two sorted lists"],
            ["Write a decorator to measure function execution time"],
        ],
        inputs=user_input,
    )

# Launch the app
print("🚀 Launching Gradio app...")
demo.launch(debug=True, share=True)

🚀 Launching Gradio app...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b357bc7f479c6313dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Starting debate for: Implement a binary search algorithm
--- Round 1 ---
🤖 Coder (GPT-4o mini) is thinking...
💎 Reviewer (Gemini) is thinking...
--- Round 2 ---
🤖 Coder (GPT-4o mini) is thinking...
💎 Reviewer (Gemini) is thinking...
--- Round 3 ---
🤖 Coder (GPT-4o mini) is thinking...
💎 Reviewer (Gemini) is thinking...
✅ Debate finished.
